<a href="https://colab.research.google.com/github/nthanhkhang/Natural-Language-Processing/blob/main/beam_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Thực hành với Beam search

Khôi phục dấu câu tiếng Việt sử dụng Beam search & Language model

> Click vào ảnh để xem bài hướng dẫn chi tiết

[![](https://nguyenvanhieu.vn/wp-content/uploads/2020/07/beam-search.jpg)](https://nguyenvanhieu.vn/thuat-toan-beam-search/)

In [1]:
import re
import json
import os
import itertools
from math import log

In [2]:
# Xóa dấu tiếng Việt
def remove_vn_accent(s):
    s = re.sub('[áàảãạăắằẳẵặâấầẩẫậ]', 'a', s)
    s = re.sub('[éèẻẽẹêếềểễệ]', 'e', s)
    s = re.sub('[óòỏõọôốồổỗộơớờởỡợ]', 'o', s)
    s = re.sub('[íìỉĩị]', 'i', s)
    s = re.sub('[úùủũụưứừửữự]', 'u', s)
    s = re.sub('[ýỳỷỹỵ]', 'y', s)
    s = re.sub('đ', 'd', s)
    return s

In [3]:
# Download VN syllables
!wget -O vn_syllables.txt "https://gist.githubusercontent.com/nguyenvanhieuvn/8d7c3440590a6db732ef6e05498c1566/raw/0164ccb7094b22a48a52844e7fa748cf2820cec9/all-vietnamese-syllables.txt(G%25C3%25B5%2520d%25E1%25BA%25A5u%2520ki%25E1%25BB%2583u%2520c%25C5%25A9)"
# Download language model
!wget "https://github.com/nguyenvanhieuvn/vn-accent-resoration/raw/master/vn_en_nextwords.txt.zip"
!unzip vn_en_nextwords.txt.zip
# Download test data
!wget "https://github.com/nguyenvanhieuvn/vn-accent-resoration/raw/master/test.txt"


'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
# Tạo bộ từ điển sinh dấu câu cho các từ không dấu
map_accents = {}
for word in open('vn_syllables.txt').read().splitlines():
  word = word.lower()
  no_accent_word = remove_vn_accent(word)
  if no_accent_word not in map_accents:
    map_accents[no_accent_word] = set()
  map_accents[no_accent_word].add(word)

FileNotFoundError: [Errno 2] No such file or directory: 'vn_syllables.txt'

In [ ]:
print(map_accents['chac'])

In [ ]:
# Đọc lm
lm = {}
for line in open('vn_en_nextwords.txt'):
  data = json.loads(line)
  key = data['s']
  lm[key] = data
vocab_size = len(lm)
total_word = 0
for word in lm:
  total_word += lm[word]['sum']
print(vocab_size, total_word)

In [ ]:
lm['khang']

In [ ]:
# tính xác suất dùng smoothing
def get_proba(current_word, next_word):
  if current_word not in lm:
    return 1 / total_word;
  if next_word not in lm[current_word]['next']:
    return 1 / (lm[current_word]['sum'] + vocab_size)
  return (lm[current_word]['next'][next_word] + 1) / (lm[current_word]['sum'] + vocab_size)

# def get_proba(current_word, next_word):
#   try:
#     return (lm[current_word]['next'][next_word]) / (lm[current_word]['sum'])
#   except:
#     return 1e-30

In [ ]:
# hàm beam search
def beam_search(words, k=3):
  sequences = []
  for idx, word in enumerate(words):
    if idx == 0:
      sequences = [([x], 0.0) for x in map_accents.get(word, [word])]
    else:
      all_sequences = []
      for seq in sequences:
        for next_word in map_accents.get(word, [word]):
          current_word = seq[0][-1]
          proba = get_proba(current_word, next_word)
          # print(current_word, next_word, proba, log(proba))
          proba = log(proba)
          new_seq = seq[0].copy()
          new_seq.append(next_word)
          all_sequences.append((new_seq, seq[1] + proba))
      # print(all_sequences) 
      all_sequences = sorted(all_sequences,key=lambda x: x[1], reverse=True)
      sequences = all_sequences[:k]
  return sequences

    

In [ ]:
# tiền xử lý text
def preprocess(sentence):
  sentence = sentence.lower()
  sentence = re.sub(r'[.,~`!@#$%\^&*\(\)\[\]\\|:;\'"]+', ' ', sentence)
  sentence = re.sub(r'\s+', ' ', sentence).strip()
  return sentence

In [ ]:
# test 1 câu
sentence = "Tuy nhiên giai đoạn từ 2009 đến ngày 17/10/2017 cong ty không còn thực hiện các  trang"
sentence = preprocess(sentence)
_sentence = remove_vn_accent(sentence)
words = _sentence.split()
results = beam_search(words, k=5)
print('INP:', sentence)

print('OUT:', ' '.join(results[0][0]))
print('CMP:', ' '.join(results[0][0]) == sentence)

INP: tuy nhiên giai đoạn từ 2009 đến ngày 175/10/2017 công ty không còn thực hiện các trang
OUT: tuy nhiên giai đoạn từ 2009 đến ngày 175/10/2017 công ty không còn thực hiện các trang
CMP: True


In [ ]:
# test qua bộ test ~ 5000 câu
k = 10
sentences = open('test.txt').read().splitlines()
test_size = len(sentences)
print(test_size)
correct = 0
for sent in sentences:
  try:
    sent = preprocess(sent)
    _sent = remove_vn_accent(sent)
    words = _sent.split()
    results = beam_search(words, k)
    if ' '.join(results[0][0]) == sent:
      correct += 1
  except:
    print('err', sent)
    break

print(correct / test_size)

  